### Bundestags-Website
- erfolgreiches Scraping von Name, Partei, Direktmandat (ja/nein), Bundesland, Wahlkreis, Ausschüssen, sozialen Medien und Biografie

### Twitter
- für CDU/CSU: 248 Abgeordnete, nur 43 Twitter Accounts über BT-Website
- Stichprobe der ersten 50 CDU/CSU-Abgeordneten: 28 haben Twitter-Account, nur wenige davon auf BT-Website gelistet
- also: händisches / semi-automatisiertes Nachtragen von Twitter-Accounts nötig (Skalierbarkeit?)
- schätzungsweise 50.000 bis 100.000 Tweets für alle CDU/CSU-Abgeordneten seit Beginn der Legislaturperiode
- bisher: Nutzung des inoffiziellen Packets GetOldTweets3 (tweepy seit Langem beantragt, noch nicht approved), keinerlei Einschränkungen für bisherige Zwecke

### Facebook
- Abfragen laufen über Facebook-interne Graph API, mit großen Einschränkungen
- nach Stand unserer bisherigen Recherchen: Facebook-Scraping nicht im selben Umfang wie Twitter-Scraping möglich

### Sozioökonomische Variablen
- wahlkreisspezifische Variablen sind (evtl. über Scraping) zugänglich

### Analyse
- Twitter-Daten können bereits ausgewertet werden (sentiment, topic)

### LRZ
- Datenspeicherung (Datenbank)
- Modellberechnungen

In [2]:
import GetOldTweets3 as got
import pandas as pd
import pickle as pickle

In [23]:
## import Bundestag data
with open('abg_df.pickle', 'rb') as handle:
    bt_data = pickle.load(handle)

In [24]:
bt_data.head()

,Name,Partei,Wahlart,Bundesland,Wahlkreis,Ausschuesse,Soziale Medien,Biografie
0,"Abercron, Dr. Michael von",CDU/CSU,Direkt gewählt,Schleswig-Holstein,Wahlkreis 007: Pinneberg,{'Ordentliches Mitglied': ['Ausschuss für Ernä...,{'von-abercron.de/': 'http://www.von-abercron....,Geboren am 17. November 1952 in Ehlers...
1,"Achelwilm, Doris",Die Linke,Gewählt über Landesliste,Bremen,n.a.,{'Ordentliches Mitglied': ['Ausschuss für Fami...,{'doris-achelwilm.de': 'http://www.doris-achel...,Geboren am 30. November 1976 in Thuine...
2,"Aggelidis, Grigorios",FDP,Gewählt über Landesliste,Niedersachsen,Wahlkreis 043: Hannover-Land I,{'Ordentliches Mitglied': ['Kuratorium der Bun...,{'grigorios-aggelidis.de': 'http://www.grigori...,Geboren am 19. August 1965 in Hannover...
3,"Akbulut, Gökay",Die Linke,Gewählt über Landesliste,Baden-Württemberg,Wahlkreis 275: Mannheim,"{'Ordentliches Mitglied': ['Schriftführer/in',...",{'goekay-akbulut.de': 'https://goekay-akbulut....,Geboren 1982 in Pinarbasi/ Türkei; ledig.Juni ...
4,"Albani, Stephan",CDU/CSU,Gewählt über Landesliste,Niedersachsen,Wahlkreis 027: Oldenburg – Ammerland,{'Ordentliches Mitglied': ['Ausschuss für Bild...,{'stephan-albani.de': 'http://www.stephan-alba...,Geboren am 3. Juni 1968 in Göttingen; verheira...


In [25]:
# create dataframe with columns 'Name' and 'Twitter Url'
def get_twitter_url(x):
    if 'Twitter' in x:
        return x['Twitter']
    else:
        return ''
url = bt_data['Soziale Medien'].apply(get_twitter_url)
twitter_account = pd.concat([bt_data['Name'], url], axis = 1, \
                            keys = ['name','url'])


In [26]:
# function to get twitter usernames from twitter url
def get_twitter_username(url):
    if url:
        return(url.split('/')[3].split('?')[0])
    else: 
        return('')

In [27]:
# function to download tweets for a specific user 
def download_tweets(username, since, until):
    print(f"Downloading for {username}")
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                               .setSince(since)\
                                               .setUntil(until)

    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    df = pd.DataFrame([tweet.__dict__ for tweet in tweets])
    return(df)


twitter_usernames = twitter_account['url'].apply(get_twitter_username)
twitter_usernames.rename("username", inplace = True)
twitter_account = pd.concat([twitter_account, twitter_usernames], axis = 1)

In [29]:
res = pd.DataFrame()
for username in twitter_account.iloc[0:3, 2]:
    res = pd.concat([res, download_tweets(username, since = "2020-02-24", \
                                          until = "2020-04-08")])

In [31]:
res = twitter_account.merge(res, on = 'username')

In [32]:
res.head()

,name,url,username,to,text,retweets,favorites,replies,id,permalink,author_id,date,formatted_date,hashtags,mentions,geo,urls
0,"Abercron, Dr. Michael von",https://twitter.com/mvabercron/,mvabercron,None,"Uni fällt aus? Keine Angst, eine Pause im Lehr...",0,0,0,1245689849627258881,https://twitter.com/mvabercron/status/12456898...,862747349277450240,2020-04-02 12:29:38+00:00,Thu Apr 02 12:29:38 +0000 2020,#BAf #Corona,,,https://cducsu.cc/3alyQN7
1,"Abercron, Dr. Michael von",https://twitter.com/mvabercron/,mvabercron,None,Alle Unternehmen können vom #Corona-Sonderprog...,2,1,1,1245431118968631297,https://twitter.com/mvabercron/status/12454311...,862747349277450240,2020-04-01 19:21:32+00:00,Wed Apr 01 19:21:32 +0000 2020,#Corona #wirhandeln,,,https://cducsu.cc/3alyQN7
2,"Abercron, Dr. Michael von",https://twitter.com/mvabercron/,mvabercron,None,Wir kämpfen um jeden Job – durch Ausweitung d....,0,0,0,1244942420841725954,https://twitter.com/mvabercron/status/12449424...,862747349277450240,2020-03-31 10:59:37+00:00,Tue Mar 31 10:59:37 +0000 2020,#wirhandeln,,,https://cducsu.cc/3alyQN7
3,"Abercron, Dr. Michael von",https://twitter.com/mvabercron/,mvabercron,None,Schnelle #Corona-Hilfe für Familien: Für den K...,0,0,0,1244646897714855939,https://twitter.com/mvabercron/status/12446468...,862747349277450240,2020-03-30 15:25:19+00:00,Mon Mar 30 15:25:19 +0000 2020,#Corona #wirhandeln,,,https://bit.ly/2Utk4OL
4,"Abercron, Dr. Michael von",https://twitter.com/mvabercron/,mvabercron,None,Der #Bundestag passt das #Infektionsschutzgese...,0,2,0,1242874119139590147,https://twitter.com/mvabercron/status/12428741...,862747349277450240,2020-03-25 18:00:56+00:00,Wed Mar 25 18:00:56 +0000 2020,#Bundestag #Infektionsschutzgesetz #Corona,,,
